***
# Evaluation of RPLH
***

In [34]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
import pandas as pd 

current_folder = Path.cwd()
parent_folder = current_folder.parent.parent
sys.path.insert(0, str(parent_folder))
print(parent_folder)

from rplh.evaluation.get_data import get_data
from rplh.evaluation.embed import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/kevinb/Desktop/dsc190/RPLH


# Importing Data

In [90]:
print("LOADING RPLH-SPY-AGENT...")
base_dir = parent_folder / "testing-env-agent"
df_a_spy, success_df_a_spy, spy_count_df, spy_df, att_df, justification_df = get_data(base_dir, 16)

print("\nLOADING RPLH-SPY-STANDARD...")
base_dir = parent_folder / "gpt-standard-testing-env-1"
df_s_spy, success_df_s_spy, _, _, _, _ = get_data(base_dir, 20)

print("\nLOADING RPLH-NoSPY-STANDADRD...")
base_dir = parent_folder / "gpt-standard-testing-nospy-env-1"
df_s_nospy, success_df_s_nospy, _, _, _, _ = get_data(base_dir, 10)

LOADING RPLH-SPY-AGENT...
trial_6: Not Converged
{'0.5_0.5': ['target_red'], '0.5_1.5': ['box_purple'], '0.5_2.5': ['target_blue'], '1.5_0.5': ['target_purple', 'box_orange'], '1.5_1.5': ['box_red', 'target_orange', 'box_blue', 'box_green', 'box_orange'], '1.5_2.5': ['target_orange', 'box_purple'], '2.5_0.5': ['target_purple'], '2.5_1.5': [], '2.5_2.5': ['target_green']}
trial_7: Not Converged
{'0.5_0.5': [], '0.5_1.5': [], '0.5_2.5': [], '1.5_0.5': [], '1.5_1.5': [], '1.5_2.5': [], '2.5_0.5': ['target_blue'], '2.5_1.5': ['box_blue'], '2.5_2.5': ['target_red', 'box_red']}
trial_15: Not Converged
{'0.5_0.5': [], '0.5_1.5': ['box_green'], '0.5_2.5': [], '1.5_0.5': ['target_green'], '1.5_1.5': [], '1.5_2.5': [], '2.5_0.5': [], '2.5_1.5': [], '2.5_2.5': []}

LOADING RPLH-SPY-STANDARD...
trial_5: Not Converged
{'0.5_0.5': [], '0.5_1.5': ['box_green'], '0.5_2.5': [], '1.5_0.5': [], '1.5_1.5': [], '1.5_2.5': [], '2.5_0.5': [], '2.5_1.5': ['target_green'], '2.5_2.5': []}
trial_9: Not Converged

In [91]:
agent_spy_merged = df_a_spy.merge(success_df_a_spy, on="Trial")
standard_spy_merged = df_s_spy.merge(success_df_s_spy, on="Trial")
standard_nospy_merged = df_s_nospy.merge(success_df_s_nospy, on="Trial")

# RPLH + Agent + Spy

In [92]:
agent_round = spy_count_df.divide(spy_count_df.sum(axis=1), axis=0).round(2)
agent_round

,6,"Agent[2.5, 0.5]","Agent[2.5, 1.5]","Agent[1.5, 1.5]",1,"Agent[0.5, 1.5]","Agent[2.5, 2.5]",0,2,5,4,"Agent[1.5, 2.5]",dummy,"Agent[0.5, 2.5]","Agent[0.5, 0.5]",3,"Agent[1.5, 0.5]"
Trial,,,,,,,,,,,,,,,,,
trial_1,0.00,0.18,0.12,0.20,0.00,0.32,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.11,0.00,0.05
trial_2,0.00,0.06,0.13,0.22,0.00,0.19,0.08,0.00,0.00,0.00,0.00,0.09,0.00,0.04,0.11,0.00,0.08
trial_3,0.00,0.11,0.15,0.10,0.00,0.17,0.17,0.00,0.00,0.00,0.00,0.07,0.00,0.02,0.11,0.00,0.12
trial_4,0.00,0.06,0.12,0.22,0.00,0.28,0.05,0.00,0.00,0.00,0.00,0.05,0.00,0.03,0.16,0.00,0.05
trial_5,0.00,0.07,0.15,0.17,0.00,0.14,0.11,0.00,0.00,0.00,0.00,0.13,0.00,0.06,0.08,0.00,0.10
trial_6,0.00,0.08,0.08,0.18,0.00,0.20,0.10,0.00,0.00,0.00,0.00,0.07,0.00,0.00,0.15,0.00,0.13
trial_7,0.00,0.08,0.12,0.16,0.00,0.18,0.12,0.00,0.00,0.00,0.00,0.08,0.00,0.05,0.11,0.00,0.09
trial_8,0.00,0.15,0.17,0.15,0.00,0.06,0.09,0.00,0.00,0.00,0.00,0.14,0.00,0.00,0.09,0.00,0.14
trial_9,0.00,0.07,0.09,0.13,0.00,0.25,0.10,0.00,0.00,0.00,0.00,0.11,0.00,0.11,0.08,0.00,0.07


In [93]:
agent_round.idxmax(axis=1).value_counts()

Agent[1.5, 1.5]    8
Agent[0.5, 1.5]    6
Agent[2.5, 1.5]    1
Agent[1.5, 0.5]    1
Name: count, dtype: int64

***
# Senetnce Embedding Analysis
***

In [94]:
# Test
sentence1 = "The cat sat on the mat."
sentence2 = "A cat was sitting on the mat."

embedding_1 = get_embedding(sentence1)
embedding_2 = get_embedding(sentence2)

cosine_sim(embedding_1, embedding_2)

0.9777018864480065

In [95]:
len(att_df['Agent[2.5, 1.5]'].explode().unique())

283

Need to check last few ones

### Compare aggregated nice with nice and nice with spy in `att_df`

In [96]:
sentence1 = "Be very easy going and nice and cooperative."
sentence2 = "".join(str(att_df['Agent[2.5, 1.5]'].explode().unique()))
sentence3 = "".join(str(att_df['Agent[2.5, 0.5]'].explode().unique()))
sentence4 = "".join(str(att_df['Agent[1.5, 1.5]'].explode().unique()))

embedding_1 = get_embedding(sentence1)
embedding_2 = get_embedding(sentence2)
embedding_3 = get_embedding(sentence3)
embedding_4 = get_embedding(sentence4)

print(f"Similarity between sentence 1 and 2 (both nice agent) is: {cosine_sim(embedding_1, embedding_2)}")
print(f"Similarity between sentence 1 and 3 (both nice agent) is: {cosine_sim(embedding_1, embedding_3)}")
print(f"Similarity between sentence 1 and 4 (4 is spy agent) is: {cosine_sim(embedding_1, embedding_4)}")
print(f"Similarity between sentence 2 and 3 (both nice agent) is: {cosine_sim(embedding_2, embedding_3)}")

Similarity between sentence 1 and 2 (both nice agent) is: 0.4579158179849494
Similarity between sentence 1 and 3 (both nice agent) is: 0.3432172138271849
Similarity between sentence 1 and 4 (4 is spy agent) is: 0.41851153370891975
Similarity between sentence 2 and 3 (both nice agent) is: 0.6923002667715256


### Compare aggregated nice with spy in `att_df`

In [97]:
sentence1 = "Be very critical and try to propose many changes"
sentence2 = "".join(str(att_df['Agent[2.5, 1.5]'].explode().unique()))
embedding_1 = get_embedding(sentence1)
embedding_2 = get_embedding(sentence2)

print(f"Similarity between sentence 1 and 2 is: {cosine_sim(embedding_1, embedding_2)}")

Similarity between sentence 1 and 2 is: 0.48430554661117997


### Compare aggregated critical with spy in `att_df`

In [98]:
sentence1 = "Be very critical and try to propose many changes"
sentence2 = "".join(str(att_df['Agent[2.5, 1.5]'].explode().unique()))
embedding_1 = get_embedding(sentence1)
embedding_2 = get_embedding(sentence2)

print(f"Similarity between sentence 1 and 2 is: {cosine_sim(embedding_1, embedding_2)}")

Similarity between sentence 1 and 2 is: 0.48430554661117997


### Compare aggregated non-spy with non-spy and spy with spy in `att_df`

In [99]:
sentence1 = "you are the spy agent, your goal is to prevent match targets"
sentence2 = "".join(str(att_df['Agent[1.5, 1.5]'].explode().unique()))
sentence3 = "".join(str(att_df['Agent[2.5, 2.5]'].explode().unique()))
sentence4 = "".join(str(att_df['Agent[2.5, 1.5]'].explode().unique()))
sentence5 = "".join(str(att_df['Agent[2.5, 0.5]'].explode().unique()))

embedding_1 = get_embedding(sentence1)
embedding_2 = get_embedding(sentence2)
embedding_3 = get_embedding(sentence3)
embedding_4 = get_embedding(sentence4)
embedding_5 = get_embedding(sentence5)

print(f"Similarity between sentence 1 and 2 (spy) is: {cosine_sim(embedding_1, embedding_2)}")
print(f"Similarity between sentence 1 and 3 (spy) is: {cosine_sim(embedding_1, embedding_3)}")
print(f"Similarity between sentence 1 and 4 (non-spy) is: {cosine_sim(embedding_1, embedding_4)}")
print(f"Similarity between sentence 1 and 5 (non-spy) is: {cosine_sim(embedding_1, embedding_5)}")

Similarity between sentence 1 and 2 (spy) is: 0.630347268530466
Similarity between sentence 1 and 3 (spy) is: 0.6390457876412771
Similarity between sentence 1 and 4 (non-spy) is: 0.6412562687371781
Similarity between sentence 1 and 5 (non-spy) is: 0.5700811775451974


### Compare spy response with spy sentence in `att_df`

In [100]:
sentence1 = "you are the spy agent, your goal is to prevent match targets"
embedding_1 = get_embedding(sentence1)
att_df['Agent[1.5, 1.5]'].apply(lambda x: cosine_sim(embedding_1, get_embedding(str(x))))

Trial
trial_1     0.501774
trial_2     0.532773
trial_3     0.528740
trial_4     0.621030
trial_5     0.541421
trial_6     0.523814
trial_7     0.563339
trial_8     0.494520
trial_9     0.484337
trial_10    0.518496
trial_11    0.638599
trial_12    0.724506
trial_13    0.379190
trial_14    0.618632
trial_15    0.635499
trial_16    0.537828
Name: Agent[1.5, 1.5], dtype: float64

### Compare spy response with spy sentence in `spy_df`

In [101]:
sentence1 = "you are the spy agent, your goal is to prevent match targets"
embedding_1 = get_embedding(sentence1)
spy_df['Agent[1.5, 1.5]'].apply(lambda x: cosine_sim(embedding_1, get_embedding(str(x))))

Trial
trial_1     0.562922
trial_2     0.596249
trial_3     0.619103
trial_4     0.724988
trial_5     0.589936
trial_6     0.552098
trial_7     0.607470
trial_8     0.529283
trial_9     0.509599
trial_10    0.474674
trial_11    0.607339
trial_12    0.700786
trial_13    0.568441
trial_14    0.666732
trial_15    0.626565
trial_16    0.520283
Name: Agent[1.5, 1.5], dtype: float64

### Compare spy response with non-spy sentence in `att_df`

In [102]:
sentence1 = "Be very easy going and nice and cooperative."
embedding_1 = get_embedding(sentence1)
att_df['Agent[1.5, 1.5]'].apply(lambda x: cosine_sim(embedding_1, get_embedding(str(x))))

Trial
trial_1     0.531023
trial_2     0.347885
trial_3     0.377885
trial_4     0.481166
trial_5     0.382660
trial_6     0.493661
trial_7     0.496129
trial_8     0.429513
trial_9     0.497625
trial_10    0.401780
trial_11    0.478452
trial_12    0.377876
trial_13    0.417905
trial_14    0.413453
trial_15    0.385275
trial_16    0.441549
Name: Agent[1.5, 1.5], dtype: float64

### Comparing in `justification_df` to see if suspecting is given

In [103]:
sentence1 = "I suspect that this agent is the spy agent, thus, I would not listen to this agent."
embedding_1 = get_embedding(sentence1)
justification_df['Justifications'].apply(lambda x: cosine_sim(embedding_1, get_embedding(str(x))))

0     0.617001
1     0.500486
2     0.650587
3     0.651606
4     0.602094
5     0.598154
6     0.640405
7     0.557793
8     0.624391
9     0.572241
10    0.659633
11    0.656388
12    0.612585
13    0.586549
14    0.671991
15    0.622219
Name: Justifications, dtype: float64

We have written a function to get everything directly

In [104]:
spy_embedding = get_spy_detect_embedding(df_a_spy, spy_df, justification_df,
                                         spy_sentence="you are the spy agent, your goal is to prevent match targets",
                                         spy_agents=["Agent[0.5, 0.5]", "Agent[1.5, 1.5]", "Agent[2.5, 2.5]"])
spy_embedding

,Trial,Num_Boxes,Boxes_To_Other,"Spy_Embed_Agent[1.5, 1.5]","Spy_Embed_Agent[2.5, 2.5]","Spy_Embed_Agent[0.5, 0.5]",Justification_Embed
0,trial_1,8,10,0.562922,0.349980,0.637555,0.616407
1,trial_2,11,37,0.596249,0.690537,0.607800,0.564882
2,trial_3,11,47,0.619103,0.573598,0.583678,0.655702
3,trial_4,10,27,0.724988,0.687081,0.634392,0.646046
4,trial_5,12,83,0.589936,0.527713,0.494512,0.628753
5,trial_6,12,10,0.552098,0.536865,0.548765,0.562042
6,trial_7,10,76,0.607470,0.568405,0.404907,0.602176
7,trial_8,8,39,0.529283,0.678788,0.612010,0.530185
8,trial_9,10,43,0.509599,0.585560,0.511685,0.605355
9,trial_10,11,25,0.474674,0.528881,0.523244,0.499253


In [105]:
spy_embedding.describe()

,Num_Boxes,Boxes_To_Other,"Spy_Embed_Agent[1.5, 1.5]","Spy_Embed_Agent[2.5, 2.5]","Spy_Embed_Agent[0.5, 0.5]",Justification_Embed
count,16.000000,16.00000,16.000000,16.000000,16.000000,16.000000
mean,10.062500,43.31250,0.591029,0.599069,0.544499,0.607042
std,1.481834,21.25941,0.068382,0.091506,0.090084,0.056144
min,8.000000,10.00000,0.474674,0.349980,0.349980,0.499253
25%,8.750000,30.00000,0.546394,0.554973,0.493548,0.564172
50%,10.000000,41.00000,0.593092,0.588626,0.557765,0.612706
75%,11.000000,52.50000,0.620969,0.679248,0.611408,0.645084
max,12.000000,83.00000,0.724988,0.690537,0.678373,0.690085


***
# Comparing Performance Across Systems
***

In [48]:
# agent_spy_merged.compare(standard_nospy_merged)

In [49]:
# agent_spy_merged.compare(standard_spy_merged)